<a href="https://colab.research.google.com/github/CaesarLCF/SoC-Estimation-Model-Development-For-Lithium-Ion-Battery-Packs/blob/main/Extended_Kalman_Filter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import math as m
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

### **1). Prepare the Datasets**

In [ ]:
# Load the battery datasets:
dataset = "battery_data.csv"      # <-- Example
df = pd.read_csv(dataset)

# Load the OCV-SoC datasets:
dataset = "OCV-SoC_data.csv"      # <-- Example
ocv = pd.read_csv(dataset)

# Prepare result columns:
df['OCV (V)'] = np.nan
df['RC Voltage (V)'] = np.nan
df['Model Voltage (V)'] = np.nan
df['Estimated SOC (%)'] = np.nan

### **2). Set up Model Parameters**




In [ ]:
# Set up model parameters:
n_EKF = 0.811                           # Coulomb counting coefficient
Q_EKF = 5.45                            # Ah (battery capacity)
R0 = 0.01                               # ohm (internal series resistance)
Rp = 0.001                              # ohm (dynamic resistance)
Cp = 30000.0                            # F (RC branch capacitance)
time_step = 1.0                         # seconds

# Define the Polynomial:
class Polynomial:
    def __init__(self, coeffs):
        self._coeffs = [float(c) for c in coeffs]
        self._deg = len(self._coeffs) - 1
    def __call__(self, x):
        x = float(x)
        value = 0.0
        xp = 1.0
        for c in self._coeffs:
            value += c * xp
            xp *= x
        return value
    @property
    def Deriv(self):
        # returns a Polynomial representing the derivative
        if self._deg <= 0:
            return Polynomial([0.0])
        d_coeffs = [(i + 1) * self._coeffs[i + 1] for i in range(self._deg)]
        return Polynomial(d_coeffs)

# Set up polynomial of OCV vs SoC:
degree = 20                                       # coefficient of polynomial

SoC = ocv['SOC (%)'].astype(float) / 100.0            # Convert from percentage to numeric
OCV = ocv['OCV (V)'].astype(float)

coefficients = np.polyfit(SoC, OCV, degree)       # Fit polynomial of OCV-SoC
coefficients_list = coefficients.tolist()[::-1]   # Low-order first -> [a0, a1, a2,...]

OCV_Model = Polynomial(coefficients_list)         # Create OCV-SoC model

### **3). Set up Extended Kalman Filter**

In [ ]:
# Initial State
x = np.matrix([[0],[0.0]])                        # x = [[SoC],[RC voltage]]; x[0,0] = SOC, x[1,0] = RC Voltage

# State Transition Model
exp_coeff = m.exp(-time_step/(Cp*Rp))
F = np.matrix([[1,0],[0,exp_coeff]])

# Control-Input Model
B = np.matrix([[-n_EKF/(Q_EKF*3600),0],[0,Rp*(1-exp_coeff)]])

# Measurement Noise Standard Deviation
std_dev = 0.00015
var = std_dev ** 2

# Convariance of Measurement Noise
R = var

# State Error Covariance
P = np.matrix([[var,0],[0,var]])

# covariance of process noise
a = 99999
Q = np.matrix([[var/a,0],[0,var/a]])

### **4). Execute Extended Kalman Filter**

In [ ]:
# Define the Extended Kalman Filter
for i in range (0,len(df)):
    u = df.at[i,'Current (A)']
    z = df.at[i,'Voltage (V)'] + R0 * df.at[i,'Current (A)']

    # Prediction Step
    x = F * x + B * u
    P = F * P * F.T + Q

    # Update Step
    H = np.matrix([[OCV_Model.Deriv(x[0,0]), -1]]) # Jacobian

    S = H * P * H.T + R
    K = P * H.T * (S)**-1

    Hx = OCV_Model(x[0,0]) - x[1,0]
    y = np.subtract(z, Hx) # Terminal Voltage Error
    x = x + K * y

    KH = K * H
    I_KH = np.identity((KH).shape[1]) - KH
    P = I_KH * P * I_KH.T + K * R * K.T

    # Estimation Step
    SOC_Estimate = x[0,0]
    df.at[i,'OCV (V)'] = OCV_Model(x[0,0])
    df.at[i,'RC Voltage (V)'] = x[1,0]
    df.at[i,'Model Voltage (V)'] = df.at[i,'OCV (V)'] - df.at[i,'RC Voltage (V)'] - R0 * u

    if SOC_Estimate * 100 < 0 :
        df.at[i,'Estimated SOC (%)'] = 0
    elif SOC_Estimate * 100 >= 100 :
        df.at[i,'Estimated SOC (%)'] = 100
    else :
        df.at[i,'Estimated SOC (%)'] = SOC_Estimate * 100

### **5). Evaluate Extended Kalman Filter**

In [ ]:
# Evaluation of EKF: SoC
print('Performance Metrics of SoC:')
# Calculate MAE
mae = mean_absolute_error(df['SOC (%)'], df['Estimated SOC (%)'])
print(f'Mean Absolute Error (MAE): {mae}')
# Calculate RMSE
mse = mean_squared_error(df['SOC (%)'], df['Estimated SOC (%)'])
rmse = (np.sqrt(mse))
print(f'Root Mean Squared Error (RMSE): {rmse}')
# Calculate R-squared
r2 = r2_score(df['SOC (%)'], df['Estimated SOC (%)'])
print(f'Coefficient of Determination (R²): {r2}')

print()

# Evaluation of EKF: Terminal Voltage
print('Performance Metrics of Terminal Voltage:')
# Calculate MAE
mae = mean_absolute_error(df['Voltage (V)'], df['Model Voltage (V)'])
print(f'Mean Absolute Error (MAE): {mae}')
# Calculate RMSE
mse = mean_squared_error(df['Voltage (V)'], df['Model Voltage (V)'])
rmse = (np.sqrt(mse))
print(f'Root Mean Squared Error (RMSE): {rmse}')
# Calculate R-squared
r2 = r2_score(df['Voltage (V)'], df['Model Voltage (V)'])
print(f'Coefficient of Determination (R²): {r2}')

In [ ]:
# Plot the data of Voltage:
plt.figure(figsize=(10, 6))
plt.plot(df['Time (s)'], df['Model Voltage (V)'], label='Model Voltage (V)', color='b', linestyle='--')
plt.plot(df['Time (s)'], df['OCV (V)'], label='OCV (V)', color='green')
plt.plot(df['Time (s)'], df['Voltage (V)'], label='Measured Voltage (V)', color='r')
plt.xlabel('Step Time (k)')
plt.ylabel('Voltage (V)')
plt.title('Voltage (V)')
plt.legend()
plt.show()

# Plot the data of SoC:
plt.figure(figsize=(10, 6))
plt.plot(df['Time (s)'], df['Estimated SOC (%)'], label='Estimated SOC (%)', color='b', linestyle='--')
plt.plot(df['Time (s)'], df['SOC (%)'], label='Actual SOC (%)', color='green')
plt.plot(df['Time (s)'], df['Predicted SOC (%)'], label='Predicted SOC (%)', color='r')
plt.xlabel('Step Time (k)')
plt.ylabel('State of Charge (%)')
plt.title('State of Charge, SoC (%)' )
plt.legend()
plt.show()

In [ ]:
# Calculate the SoC Error:
df['SOC Error'] = df['Estimated SOC (%)'] - df['SOC (%)']

# Plot the SoC Error:
plt.figure(figsize=(10, 6))
plt.plot(df['Time (s)'], df['SOC Error'], label='Prediction Error', color='m')
plt.axhline(y=0, color='k', linestyle='--')
plt.xlabel('Step Time (k)')
plt.ylabel('Error (Actual SOC - Estimated SOC)')
plt.title('Prediction Error Over Time')
plt.legend()
plt.show()

# Calculate the Voltage Error:
df['Volatge Error'] = df['Model Voltage (V)'] - df['Voltage (V)']

# Plot the errors
plt.figure(figsize=(10, 6))
plt.plot(df['Time (s)'], df['Volatge Error'], label='Prediction Error', color='m')
plt.axhline(y=0, color='k', linestyle='--')
plt.xlabel('Step Time (k)')
plt.ylabel('Error (Model Voltage - Measured Voltage)')
plt.title('Prediction Error Over Time')
plt.legend()
plt.show()